In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pdb
import argparse
import numpy as np
import matplotlib.pyplot as plt
import scipy 
import sys
import time
import glob
import pickle
import pandas as pd
from tqdm import tqdm
from sklearn.linear_model import LinearRegression

import itertools
from sklearn.model_selection import KFold

sys.path.append('/home/akumar/nse/neural_control')
from utils import apply_df_filters, calc_loadings
from loaders import load_sabes
from segmentation import reach_segment_sabes
from decoders import lr_decode_windowed

from mpi4py import MPI
from copy import deepcopy

/home/akumar/anaconda3/envs/ncontrol/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Debug the decoding vs t. Why does there seem to be variability across runs?

In [3]:
sys.path.append('/home/akumar/nse/neural_control/analysis_scripts')

In [4]:
from decodingvt_cv_ilmerge import filter_reach_type
from decodingvt_cv_strvcorr import behavioral_metrics

In [5]:
#dimvals = np.array([2, 6, 10, 15])
# Fix dimension to 6
dimval = 6
measure_from_end=False

# Sliding windows
window_width = 2
#window_centers = np.linspace(0, 35, 25)[0:9]
window_centers = np.arange(20)
windows = [(int(wc - window_width//2), int(wc + window_width//2)) for wc in window_centers]

# with open('/home/akumar/nse/neural_control/data/indy_dimreduc_marginal_nocv.dat', 'rb') as f:
#     sabes_df = pickle.load(f)
# with open('/mnt/Secondary/data/postprocessed/indy_dimreduc_nocv.dat', 'rb') as f:
#     sabes_df = pickle.load(f)
with open('/mnt/Secondary/data/postprocessed/indy_decoding_df2.dat', 'rb') as f:
    indy_df = pickle.load(f)
indy_df = pd.DataFrame(indy_df)

with open('/mnt/Secondary/data/postprocessed/loco_decoding_df.dat', 'rb') as f:
    loco_df = pickle.load(f)
loco_df = pd.DataFrame(loco_df)
loco_df = apply_df_filters(loco_df,
                        loader_args={'bin_width': 50, 'filter_fn': 'none', 'filter_kwargs': {}, 'boxcox': 0.5, 'spike_threshold': 100, 'region': 'M1'},
                        decoder_args={'trainlag': 4, 'testlag': 4, 'decoding_window': 5})
good_loco_files = ['loco_20170210_03.mat',
'loco_20170213_02.mat',
'loco_20170215_02.mat',
'loco_20170227_04.mat',
'loco_20170228_02.mat',
'loco_20170301_05.mat',
'loco_20170302_02.mat']

loco_df = apply_df_filters(loco_df, data_file=good_loco_files)        

# with open('/home/akumar/nse/neural_control/data/sabes_dimreduc_nocv.dat', 'rb') as f:
#     sabes_df = pickle.load(f)

sabes_df = pd.concat([indy_df, loco_df])

data_files = np.unique(sabes_df['data_file'].values)

/home/akumar/anaconda3/envs/ncontrol/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 0.24.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/akumar/anaconda3/envs/ncontrol/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 0.24.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
def load_and_segment(didx):
    data_file = data_files[didx]

    # df = apply_df_filters(sabes_df, data_file=data_file, dim=dimval)
    # assert(df.shape[0] == 1)
    # coefpca.append(df.iloc[0]['pcacoef'])
    # coeffcca.append(df.iloc[0]['lqgcoef'])            

    dffca = apply_df_filters(sabes_df, data_file=data_file, dim=dimval, dimreduc_method='LQGCA', dimreduc_args={'T':3, 'loss_type':'trace', 'n_init':10})
    dfpca = apply_df_filters(sabes_df, data_file=data_file, dim=dimval, dimreduc_method='PCA')

    assert(dffca.shape[0] == 5)
    assert(dfpca.shape[0] == 5)

    coefpca = [apply_df_filters(dfpca, fold_idx=k).iloc[0]['coef'][:, 0:dimval] for k in range(5)]
    coeffcca = [apply_df_filters(dffca, fold_idx=k).iloc[0]['coef'][:, 0:dimval] for k in range(5)]

    dat = load_sabes('/mnt/Secondary/data/sabes/%s' % data_file)
    # dat = load_sabes(data_file)
    data_file = data_file.split('/')[-1]
    # dat = load_sabes('/mnt/sdb1/nc_data/sabes/%s' % data_file)
    dat = reach_segment_sabes(dat, data_file=data_file.split('.mat')[0])
    X = np.squeeze(dat['spike_rates'])
    Z = dat['behavior']
    # transition_times = dat['transition_times']


    reach_filter = 0
    error_thresh=1.00
    error_op = 'le'
    filter_op = 'le'
    q = 0.00

    transition_times, error_filter, reach_filter, window_filter = filter_reach_type(dat, reach_filter, 
                                                                                    error_thresh, error_op, 
                                                                                    q=q, op=filter_op, windows=windows)
    # Encode the error_thresh, error_op, reach filter, q and op into a string

    target_pairs = dat['target_pairs']
    return X, Z, transition_times, target_pairs, coefpca, coeffcca

In [46]:
list(KFold(n_splits=5).split(Z))

[(array([ 9133,  9134,  9135, ..., 45662, 45663, 45664]),
  array([   0,    1,    2, ..., 9130, 9131, 9132])),
 (array([    0,     1,     2, ..., 45662, 45663, 45664]),
  array([ 9133,  9134,  9135, ..., 18263, 18264, 18265])),
 (array([    0,     1,     2, ..., 45662, 45663, 45664]),
  array([18266, 18267, 18268, ..., 27396, 27397, 27398])),
 (array([    0,     1,     2, ..., 45662, 45663, 45664]),
  array([27399, 27400, 27401, ..., 36529, 36530, 36531])),
 (array([    0,     1,     2, ..., 36529, 36530, 36531]),
  array([36532, 36533, 36534, ..., 45662, 45663, 45664]))]

In [85]:
didxs = np.arange(1)

lag = 4
decoding_window = 5

# Distribute windows across ranks
wr2 = np.zeros((35, len(windows), 5, 2, 6))
ntr = np.zeros((35, len(windows), 5, 2))

# Apply projection
MSEtr = np.zeros((35, len(windows), 5, 2), dtype=object)
MSEte = np.zeros((35, len(windows), 5, 2), dtype=object)

full_reaches_train = np.zeros((35, len(windows), 5), dtype=object)
full_reaches_test = np.zeros((35, len(windows), 5), dtype=object)

# Windows x folds x straight/corrective x train/test x metric
behavioral_metrics_array = np.zeros((35, len(windows), 5, 2, 3), dtype=object)

# Cross-validate the prediction
for ii, didx in tqdm(enumerate(didxs)):
    X, Z, transition_times, target_pairs, coefpca, coeffcca = load_and_segment(didx)
    for j, window in enumerate(windows):
        for fold, (train_idxs, test_idxs) in enumerate(KFold(n_splits=5).split(Z)): 
            xpca = X @ coefpca[fold]
            xfcca = X @ coeffcca[fold]

            # Need to turn train/test_idxs returned by KFold into an indexing of the transition times
            tt_train_idxs = [idx for idx in range(len(transition_times)) if transition_times[idx][0] in train_idxs and transition_times[idx][1] in train_idxs]
            tt_test_idxs = [idx for idx in range(len(transition_times)) if transition_times[idx][0] in test_idxs and transition_times[idx][1] in test_idxs]

            r2pos, r2vel, r2acc, r2post, r2velt, r2acct, reg, ntr_, fitr, fite, msetr, msete  = lr_decode_windowed(xpca, Z, lag, [window], [window], transition_times, train_idxs=tt_train_idxs,
                                                                                                                    test_idxs=tt_test_idxs, decoding_window=decoding_window)
            # Narrow down the msetr and msete by the fitr/fite 
            try:
                msetr = [msetr[i] for i in range(len(msetr)) if i in fitr]
                msete = [msete[i] for i in range(len(msete)) if i in fite]
            except:
                msetr = np.nan
                msete = np.nan

            wr2[ii, j, fold, 0, :] = (r2pos, r2vel, r2acc, r2post, r2velt, r2acct)
            MSEtr[ii, j, fold, 0] = msetr
            MSEte[ii, j, fold, 0] = msete

            # Save for comparison
            with open('test5w%dfold%d.dat' % (j, fold), 'wb') as f:
                f.write(pickle.dumps(tt_train_idxs))
                f.write(pickle.dumps(tt_test_idxs))
                f.write(pickle.dumps(ntr_))
                f.write(pickle.dumps(fitr))
                f.write(pickle.dumps(fite))
                f.write(pickle.dumps(wr2))
                f.write(pickle.dumps(msetr))
                f.write(pickle.dumps(msete))

            r2pos, r2vel, r2acc, r2post, r2velt, r2acct, reg, ntr_, fitr, fite, msetr, msete  = lr_decode_windowed(xfcca, Z, lag, [window], [window], transition_times, train_idxs=tt_train_idxs,
                                                                                                                    test_idxs=tt_test_idxs, decoding_window=decoding_window)

            wr2[ii, j, fold, 1, :] = (r2pos, r2vel, r2acc, r2post, r2velt, r2acct)
            # Narrow down the msetr and msete by the fitr/fite 
            try:
                msetr = [msetr[i] for i in range(len(msetr)) if i in fitr]
                msete = [msete[i] for i in range(len(msete)) if i in fite]
            except:
                msetr = np.nan
                msete = np.nan

            MSEtr[ii, j, fold, 1] = msetr
            MSEte[ii, j, fold, 1] = msete

            # Convert fitr and fite to an indexing of the original transition times
            try:
                fitr = [tt_train_idxs[idx] for idx in fitr]
                fite = [tt_test_idxs[idx] for idx in fite]
            except:
                fitr = np.nan
                fite = np.nan

            full_reaches_train[ii, j, fold] = fitr
            full_reaches_test[ii, j, fold] = fite

            # Calculate behavioral metrics
            dftsecondphase_tr, maxperpd_tr, secondphaseduration_tr = behavioral_metrics(fitr, np.array(transition_times), target_pairs, deepcopy(Z))
            dftsecondphase_te, maxperpd_te, secondphaseduration_te = behavioral_metrics(fite, np.array(transition_times), target_pairs, deepcopy(Z))

            behavioral_metrics_array[ii, j, fold, 0, 0] = dftsecondphase_tr
            behavioral_metrics_array[ii, j, fold, 0, 1] = maxperpd_tr
            behavioral_metrics_array[ii, j, fold, 0, 2] = secondphaseduration_tr

            behavioral_metrics_array[ii, j, fold, 1, 0] = dftsecondphase_te
            behavioral_metrics_array[ii, j, fold, 1, 1] = maxperpd_te
            behavioral_metrics_array[ii, j, fold, 1, 2] = secondphaseduration_te


0it [00:00, ?it/s]

Reminder that start times depend on the bin size
724 Reaches


/home/akumar/nse/neural_control/segmentation.py:68: RuntimeWarning: divide by zero encountered in double_scalars
  m = straight[1]/straight[0]
/home/akumar/nse/neural_control/analysis_scripts/decodingvt_cv_strvcorr.py:258: RuntimeWarning: invalid value encountered in divide
  velpks, _ = scipy.signal.find_peaks(vel_/np.max(vel_), height=0.4, prominence=0.1)
/home/akumar/nse/neural_control/analysis_scripts/decodingvt_cv_strvcorr.py:94: RuntimeWarning: divide by zero encountered in double_scalars
  m = straight[1]/straight[0]
/home/akumar/nse/neural_control/analysis_scripts/decodingvt_cv_strvcorr.py:258: RuntimeWarning: invalid value encountered in divide
  velpks, _ = scipy.signal.find_peaks(vel_/np.max(vel_), height=0.4, prominence=0.1)
/home/akumar/nse/neural_control/analysis_scripts/decodingvt_cv_strvcorr.py:94: RuntimeWarning: divide by zero encountered in double_scalars
  m = straight[1]/straight[0]
/home/akumar/nse/neural_control/analysis_scripts/decodingvt_cv_strvcorr.py:258: Run

In [19]:
wr2_ref = deepcopy(wr2)
MSEtr_ref = deepcopy(MSEtr)
MSEte_ref = deepcopy(MSEte)

In [22]:
np.allclose(wr2, wr2_ref)

False

In [23]:
wr2_ref

array([[[[[ 3.32228247e-01,  2.04128576e-01,  7.66754713e-02,
            2.09137300e-01,  2.71838801e-03, -8.61747252e-02],
          [ 4.75130708e-01,  2.34519963e-01,  8.85182166e-02,
            3.65045285e-01,  1.26612273e-02, -5.57268515e-02]],

         [[ 2.94087100e-01,  1.71835307e-01,  5.95544126e-02,
            2.93328824e-01,  1.43323410e-01,  1.11109730e-02],
          [ 4.59307653e-01,  2.19043447e-01,  8.49642305e-02,
            4.04771424e-01,  1.37835375e-01, -4.20899362e-02]],

         [[ 2.85288403e-01,  1.51674710e-01,  5.82979757e-02,
            3.38586596e-01,  1.79559626e-01,  1.37444043e-02],
          [ 4.30136272e-01,  1.90601851e-01,  7.59321203e-02,
            4.93671020e-01,  2.25692130e-01,  2.54468457e-02]],

         [[ 3.36234833e-01,  1.92967639e-01,  6.65935272e-02,
            1.78742723e-01,  5.32024469e-02,  2.01361394e-03],
          [ 4.63491481e-01,  2.33311095e-01,  7.95795690e-02,
            3.57460935e-01,  1.07102692e-01,  1.91629679e

In [25]:
wr2.shape

(35, 20, 5, 2, 6)

In [31]:
wr2[0, 0, 1, ...]

array([[ 0.17871188,  0.04667537,  0.05024139,  0.14532528, -0.0160847 ,
        -0.01388668],
       [ 0.25694923,  0.03787716,  0.04040812,  0.21746066,  0.00365252,
         0.00849663]])

In [32]:
wr2_ref[0, 0, 1, ...]

array([[ 0.2940871 ,  0.17183531,  0.05955441,  0.29332882,  0.14332341,
         0.01111097],
       [ 0.45930765,  0.21904345,  0.08496423,  0.40477142,  0.13783538,
        -0.04208994]])

In [51]:
MSEte[0][0, 1, 0]

[array([[1281.73142704,  431.44587172,  209.49474126,   70.18608442,
           27.93323356,  229.17263005],
        [1725.32647941, 1539.38364367,  123.7268846 ,  185.81142739,
          407.02179613, 2818.81566562]]),
 array([[13206.42001568,   498.50600866,  4253.35715994,  2274.46259556,
          2407.76848006,   362.0840774 ],
        [11756.15642749,   330.27222962,  2817.95063391,  5633.28409188,
          1339.94655562,  2977.59729933]]),
 array([[ 3569.07889195,   184.13138309,  4080.79732823,  7180.52664423,
          9887.82526323, 14368.8634167 ],
        [ 2954.70322326,   401.28515649,  3158.60195422,  4387.5180195 ,
          4980.45033644,  8283.41601627]]),
 array([[6218.66066022, 1215.33551795,   73.14548453, 2464.00527761,
         1660.23008045, 6605.86430206],
        [4934.99371623, 1403.24026884,  137.13351315, 1604.56780264,
         1036.93142321, 3994.66127443]]),
 array([[11423.98824148,  2170.40626547,    16.84176237,  2811.38579131,
           515.94932413

In [52]:
MSEte_ref[0][0, 1, 0]

[array([[2.46454321e+03, 2.18255475e+01, 1.06149680e-02, 4.69789407e-01,
         4.52934336e-03, 2.04280188e-01],
        [3.09292181e+03, 1.31640385e+01, 1.57654548e-02, 4.10823415e-01,
         1.35761830e-03, 1.89972402e-01]]),
 array([[5.03294823e+03, 1.90841319e+03, 1.61623549e-03, 1.18381413e+00,
         3.42946321e-03, 1.19212507e-03],
        [4.38250827e+03, 2.66010632e+03, 3.40431909e-03, 1.15073409e+00,
         3.04876640e-03, 6.25342741e-04]]),
 array([[6.26106647e+03, 3.00319075e+03, 1.35868743e-03, 1.17457377e-01,
         7.46800463e-04, 2.38487198e-03],
        [5.17492781e+03, 3.01739749e+03, 1.67399828e-03, 1.20043771e-01,
         7.89959422e-03, 4.15304277e-03]]),
 array([[9.70289068e+03, 3.09292868e+02, 1.88458881e-03, 6.00412405e-03,
         8.91158326e-03, 1.23938791e-02],
        [7.87751281e+03, 4.98476809e+02, 5.96222962e-03, 2.14296682e-03,
         2.06818508e-03, 1.91973151e-03]]),
 array([[7.73971652e+03, 1.85154633e+02, 4.27717581e-03, 3.01543982e-03,

In [88]:
with open('test5w0fold1.dat', 'rb') as f:
    trainidx1 = pickle.load(f)
    testidxs1 = pickle.load(f)
    ntr1 = pickle.load(f)
    fitr1 = pickle.load(f)
    fite1 = pickle.load(f)
    r21 = pickle.load(f)
with open('test4w0fold1.dat', 'rb') as f:
    trainidx2 = pickle.load(f)
    testidxs2 = pickle.load(f)
    ntr2 = pickle.load(f)
    fitr2 = pickle.load(f)
    fite2 = pickle.load(f)
    r22 = pickle.load(f)

In [78]:
np.allclose(trainidx1, trainidx2)

True

In [79]:
np.allclose(testidxs1, testidxs2)

True

In [80]:
np.allclose(fite1, fite2)

True

In [89]:
np.allclose(r21, r22)

False

In [ ]:
# Conclusion: need to use deepcopy when using calculating behavioral metrics